# Submission by: Eoin O'Reilly - 2930768, Neil O'Brien - 2918015

## Summary of Division of Work
* Student 1 Name & Number: Eoin O'Reilly 2930768					
* Student 2 Name & Number: Neil O'Brien 2918015

**Division of work:** work was evenly divided 


### Code repository 
https://github.com/eoinoreilly/Draughts

### Percentage of work completed by each partner on each class / task 

| Class name/Task 	|   Eoin O'Reilly 	|    Neil O'Brien 	|
|--------------------------------	|----------------	|----------------	|
| Draughts 	        |  50%  |  50% 	|
| Board 	        |  50% 	|  50%	|
| Piece 	        |  50%	|  50%	|
| System Design 	 |  50%	 |  50%  |
| Git hub repository 	|  70%	|  30% 	|
| Learning the rules of Draughts 	|  50%	|  50%	|
| Writing the rules in logic  	|  30% 	|  70%	|


 


## Solution (tasks 1 to 9)
Not all requirements fulfilled. Highlighting of available moves, consecutive captures and crowning of pieces that reach the end of the board (and subsequent additional mobility of that piece) are not implemented.

In [2]:
'''__main__.py'''

from PyQt5.QtWidgets import QApplication
from draughts import Draughts
import sys

app = QApplication([])
draughts = Draughts()
sys.exit(app.exec_())

ModuleNotFoundError: No module named 'draughts'

In [3]:
'''board.py'''

from PyQt5.QtCore import Qt, QBasicTimer, pyqtSignal, QPoint, pyqtSlot
from PyQt5.QtGui import QPainter, QColor
from PyQt5.QtWidgets import QFrame, QLabel, QMessageBox
from piece import Piece
from player import Player


class Board(QFrame):
    msg2StatusBar = pyqtSignal(str)
    updateTimerSignal = pyqtSignal(int) # signal sent when timer is updated
    updateActivePlayer = pyqtSignal(str)  # signal sent when there is a new active player
    updateScore = pyqtSignal(str)

    boardWidth = 8
    boardHeight = 8
    timerSpeed = 1000  # the timer updates ever 1 second

    def __init__(self, parent):
        super().__init__(parent)
        self.init_board()

    def init_board(self):
        # initiates board
        self.timer1 = QBasicTimer()
        self.timer2 = QBasicTimer()
        self.counter1 = 300  # the number the counter will count down from
        self.counter2 = 300
        self.isWaitingAfterLine = False
        self.setFocusPolicy(Qt.StrongFocus)
        self.isStarted = False
        self.isPaused = False
        self.clicks = 0
        self.selectedPiece, self.fromRow, self.fromCol = 0, 0, 0
        self.currentSquare = ()
        self.selectedSquare = ()
        self.pieceCaptured = False
        self.currentPlayer = Player.Player1 # Defaut Start
        self.turn = 0
        self.pieceSelected = False
        self.updateActivePlayer.emit(self.currentPlayer.name)

        self.boardArray = [
            [0, 1, 0, 1, 0, 1, 0, 1],
            [1, 0, 1, 0, 1, 0, 1, 0],
            [0, 1, 0, 1, 0, 1, 0, 1],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [2, 0, 2, 0, 2, 0, 2, 0],
            [0, 2, 0, 2, 0, 2, 0, 2],
            [2, 0, 2, 0, 2, 0, 2, 0],
        ]
        self.start()  # start the game which will start the timer

    def paintEvent(self, event):
        # paints the board and the pieces of the game
        painter = QPainter(self)
        # rect = self.contentRect()
        self.draw_grid(painter)
        self.draw_pieces(painter)

    def timerEvent(self, event):
        # this event is automatically called when the timer is updated. based on the timerSpeed variable
        # todo adapt this code to handle your timers
        if event.timerId() == self.timer1.timerId():  # if the timer that has 'ticked' is the one in this class
            if self.counter1 == 0:
                self.win_state("Player2")
                self.timer1.stop()

            self.counter1 = self.counter1 - 1
            self.updateTimerSignal.emit(self.counter1)

        elif event.timerId() == self.timer2.timerId():  # if the timer that has 'ticked' is the one in this class
            if self.counter2 == 0:
                self.win_state("Player1")
                self.timer2.stop()

            self.counter2 = self.counter2 - 1
            self.updateTimerSignal.emit(self.counter2)

        else:
            super(Board, self).timerEvent(event)  # other wise pass it to the super class for handling

    def mousePressEvent(self, event):
        '''
        Main game play is contained here.  Player turns, game play flags, scores
        and captures are all part of the mousePressEvent.
        '''
        row, col = self.mouse_pos_to_col_row(event)
        self.currentSquare = (row, col)

        # Set expected player and expected player piece
        if self.player_turn() == 0:  # We start with Player 1
            self.currentPlayer = Player.Player1
            player_piece = Piece.Blue

        else:
            self.currentPlayer = Player.Player2
            player_piece = Piece.Red

        # If player begins by selecting an empty square, create error pop up
        if self.clicks == 0 and self.get_pieces(row, col) == 0:
            QMessageBox.about(self, "Error", "You must select a piece first")
            return

        # Ensure Player selects their own piece
        if self.pieceSelected == False and ((self.currentPlayer.name == 'Player1' and self.boardArray[row][col] != 1)
          or (self.currentPlayer.name == 'Player2' and self.boardArray[row][col] != 2)):
            QMessageBox.about(self, "Error", "{} must select a {} piece".format(self.currentPlayer.name, player_piece.name))
            return

        # For the fist click of each player, we store the various parameters
        # for use later in the method.  Here we also highlight the selected
        # piece
        if self.clicks == 0:
            self.clicks += 1
            self.fromRow, self.fromCol = row, col
            self.selectedSquare = (row, col)
            self.selectedPiece = self.boardArray[row][col]
            self.boardArray[row][col] = 3
            self.pieceSelected = True

        # Allow player to deselect a square
        elif self.pieceSelected and (self.selectedSquare == self.currentSquare):
            self.boardArray[row][col] = self.selectedPiece
            self.pieceSelected = False
            self.clicks = 0

        # After Player has selected a piece, we parse the game play logic to
        # dtermine valid moves, captured pieces and scoring
        elif self.clicks >= 1:
            self.clicks += 1

            # We check here if it's a valid move, do not proceed if not.
            #
            if self.is_valid_move(self.selectedSquare, self.currentSquare, self.currentPlayer):
                # We move the selected peice to the target row/col in the array
                # and mark the source square as empty (0)
                self.boardArray[row][col] = self.selectedPiece
                self.boardArray[self.fromRow][self.fromCol] = 0

                # Remove the captured piece, opposite logic for each player
                if self.pieceCaptured:
                    if self.currentPlayer.name == 'Player1':
                        # Update the Player score
                        self.updateScore.emit(self.currentPlayer.name)
                        # We've moved to the right of the board
                        if col > self.fromCol:
                            self.boardArray[row - 1][col - 1] = 0
                        else:
                            self.boardArray[row - 1][col + 1] = 0

                    if self.currentPlayer.name == 'Player2':
                        # Update the Player score
                        self.updateScore.emit(self.currentPlayer.name)
                        # We've moved to the right of the board
                        if col > self.fromCol:
                            self.boardArray[row + 1][col - 1] = 0
                        else:
                            self.boardArray[row + 1][col + 1] = 0
                # Reset flags
                self.pieceCaptured = False
                self.clicks = 0
                self.turn += 1

                if self.currentPlayer.name == 'Player1':
                    self.timer1.stop()
                    self.updateActivePlayer.emit('Player2')
                    self.msg2StatusBar.emit("Player2, it's your turn!")
                    self.timer2.start(Board.timerSpeed, self)

                else:
                    self.timer2.stop()
                    self.updateActivePlayer.emit('Player1')
                    self.msg2StatusBar.emit("Player1, it's your turn!")
                    self.timer1.start(Board.timerSpeed, self)

                # Reset flag for next player
                self.pieceSelected = False
            else:
                QMessageBox.about(self, "Error", "{} invalid move".format(self.currentPlayer.name))
        # Update the board
        self.update()

    def draw_grid(self, painter):
        # draw all the squares on the board
        # todo set the default colour of the brush
        for row in range(0, Board.boardHeight):
            for col in range(0, Board.boardWidth):
                if (row + col) % 2 == 0:
                    colour = QColor(255, 255, 204)

                else:
                    colour = QColor(204, 255, 204)

                painter.save()
                painter.translate(col * self.square_width(), row * self.square_height())
                painter.setBrush(colour)
                painter.fillRect(0, 0, self.square_width(), self.square_height(), colour)
                painter.restore()

    def draw_pieces(self, painter):
        # draw the prices on the board
        for row in range(0, len(self.boardArray)):
            for col in range(0, len(self.boardArray[0])):
                painter.restore()
                painter.save()
                painter.translate(col * self.square_width(), row * self.square_height())

                if self.boardArray[row][col] == 1:
                    colour = QColor(128, 179, 255)

                elif self.boardArray[row][col] == 2:
                    colour = QColor(255, 128, 128)

                elif self.boardArray[row][col] == 3:
                    colour = QColor(244, 170, 66)

                else:
                    continue

                painter.setBrush(colour)

                if self.boardArray[row][col] > 0:
                    radius_x = int(self.square_height() / 2)
                    radius_y = int(self.square_width() / 2)
                    center = QPoint(radius_y, radius_x)
                    painter.drawEllipse(center, 9 * radius_y / 10, 9 * radius_x / 10)

    def square_width(self):
        # returns the width of one square in the board
        return self.contentsRect().width() / Board.boardWidth

    def square_height(self):
        # returns the height of one square in the board
        return self.contentsRect().height() / Board.boardHeight

    def print_board_array(self):
        # prints the boardArray in an attractive way
        print("boardArray:")
        print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in self.boardArray]))

    def player_turn(self):
        return self.turn % 2

    def mouse_pos_to_col_row(self, event):
        # convert the mouse click event to a row and column
        self.click_row = int(event.y() / self.square_height())
        self.click_col = int(event.x() / self.square_width())
        return self.click_row, self.click_col

    def start(self):
        # starts game
        if self.isPaused:
            return

        self.isStarted = True
        self.isWaitingAfterLine = False
        self.numLinesRemoved = 0
        self.msg2StatusBar.emit("{}, it's your turn!".format(self.currentPlayer.name))
        self.timer1.start(Board.timerSpeed, self)

    def pause(self):
        # pauses and resumes game

        if not self.isStarted:
            return

        self.isPaused = not self.isPaused

        if self.isPaused:
            self.timer1.stop()
            self.timer2.stop()
            self.msg2StatusBar.emit("Paused on {}'s turn.".format(self.currentPlayer.name))

        else:
            if self.currentPlayer.name == 'Player1':
                self.timer1.start(Board.timerSpeed, self)

            elif self.currentPlayer.name == 'Player2':
                self.timer2.start(Board.timerSpeed, self)

            self.msg2StatusBar.emit("{}, it's your turn!".format(self.currentPlayer.name))
        self.update()

    def make_connection(self, draughts):
        draughts.resetGame.connect(self.reset)

    @pyqtSlot()
    def reset(self):
        '''
        Reset Game
        '''
        self.init_board()
        self.update()

    def opponent_adjacent(self, player, current_square):
        '''
        Check if there is an opponent diagonally adjacent
        '''
        if player.name == 'Player1':
            # Only look to the left column if we're on the right edge of the board
            if current_square[1] == 7:
                if self.boardArray[current_square[0] + 1][6] == 2:
                    return True
                return False
            # Only look to the right column if we're on the left edge of the board
            if current_square[1] == 0:
                if self.boardArray[current_square[0] + 1][1] == 2:
                    return True
                return False
            if (self.boardArray[current_square[0] + 1][current_square[1] - 1] == 2
                or self.boardArray[current_square[0] + 1][current_square[1] + 1] == 2
                ):
                return True
        if player.name == 'Player2':
            # Only look to the left column if we're on the right edge of the board
            if current_square[1] == 7:
                if self.boardArray[current_square[0] - 1][6] == 1:
                    return True
                return False
            # Only look to the right column if we're on the left edge of the board
            if current_square[1] == 0:
                if self.boardArray[current_square[0] - 1][1] == 1:
                    return True
                return False
            if (self.boardArray[current_square[0] - 1][current_square[1] - 1] == 1
                or self.boardArray[current_square[0] - 1][current_square[1] + 1] == 1
                ):
                return True
        return False

    def is_valid_move(self, from_square, to_square, player):
        '''
        Valid moves will change depending on whether there is an opponent
        adjacent or not, or if the target square is empty or not.
        '''
        opponent_adj = self.opponent_adjacent(player, from_square)
        if player.name == 'Player1':
            # If there's no adjacent opponent, valid move is 1 diagonal forward
            # as long as destination has no piece
            if not opponent_adj:
                if ((((from_square[0] + 1, from_square[1] + 1) == to_square)
                  or (from_square[0] + 1, from_square[1] - 1) == to_square)
                  and self.get_pieces(to_square[0], to_square[1]) == 0):
                    return True

            # If we have an adjacent opponent, we can move 1 diagonal forward
            # or 2 diagonal forward as long as destination has no piece
            if opponent_adj:
                if ((((from_square[0] + 2, from_square[1] + 2)  == to_square)
                  or ((from_square[0] + 2, from_square[1] - 2) == to_square))
                  and self.get_pieces(to_square[0], to_square[1]) == 0):
                    self.pieceCaptured = True
                    return True

                if ((((from_square[0] + 1, from_square[1] + 1) == to_square)
                  or ((from_square[0] + 1, from_square[1] - 1) == to_square))
                  and self.get_pieces(to_square[0], to_square[1]) == 0):
                    self.pieceCaptured = False
                    return True

            else:
                return False

        if player.name == 'Player2':
            # If there's no adjacent opponent, valid move is 1 diagonal forward
            # as long as destination has no piece
            if not opponent_adj:
                if ((((from_square[0] - 1,from_square[1] + 1) == to_square)
                  or (from_square[0] - 1, from_square[1] - 1) == to_square)
                  and self.get_pieces(to_square[0], to_square[1]) == 0):
                    return True

            # If we have an adjacent opponent, we can move 1 diagonal forward
            # or 2 diagonal forward as long as destination has no piece
            if opponent_adj:
                if ((((from_square[0] - 2,from_square[1] + 2)  == to_square)
                  or ((from_square[0] - 2, from_square[1] - 2) == to_square))
                  and self.get_pieces(to_square[0], to_square[1]) == 0):
                    self.pieceCaptured = True
                    return True

                if ((((from_square[0] - 1,from_square[1] + 1) == to_square)
                  or ((from_square[0] - 1, from_square[1] - 1) == to_square))
                  and self.get_pieces(to_square[0], to_square[1]) == 0):
                    self.pieceCaptured = False
                    return True

            else:
                return False

        return False

    def get_pieces(self, row, col):
        if col < 0 or col > 7 or row < 0 or row > 7:
            return False

        else:
            return self.boardArray[row][col]

    def win_state(self, player):
        # Produces a message that pops up when a player enters a win state
        # A win sate is entered when all a player's pieces are captured or a player's timer reaches zero
        # The message has a reset button that will start a new game
        self.msg2StatusBar.emit("{}, you win!".format(player))
        win_msg = QMessageBox()
        win_msg.setIcon(QMessageBox.Information)
        win_msg.setInformativeText("Congratulations!\n{} is the Winner".format(player))
        win_msg.setWindowTitle("Winner")
        win_msg.addButton(QMessageBox.Reset)
        win_msg.buttonClicked.connect(self.reset)
        win_msg.exec_()


    def connection(self, scoreBoard):
        # this handles a signal sent from the scoreBoard class
        # when the winnerSignal is emitted in scoreBoard the win_state slot receives it
        scoreBoard.winnerSignal.connect(self.win_state)



ModuleNotFoundError: No module named 'piece'

In [4]:
'''draughts.py'''

import sys

from PyQt5.QtCore import Qt, pyqtSignal
from PyQt5.QtWidgets import QMainWindow, QDesktopWidget, QApplication, QLabel, QAction, QMessageBox, QPushButton
from PyQt5.QtGui import QIcon
from board import Board
from scoreBoard import ScoreBoard


class Draughts(QMainWindow):
    resetGame = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        # initiates application UI
        self.tboard = Board(self)
        self.setCentralWidget(self.tboard)

        self.scoreBoard = ScoreBoard()
        self.scoreBoard.setAllowedAreas(Qt.RightDockWidgetArea | Qt.LeftDockWidgetArea)
        self.addDockWidget(Qt.RightDockWidgetArea, self.scoreBoard)

        self.reset_button = QPushButton(QIcon("../img/reset.png"), "", self)
        self.reset_button.setToolTip("Reset the game")
        self.reset_button.clicked.connect(self.reset)

        self.pause_button = QPushButton(QIcon("../img/pause_play.png"), "", self)
        self.pause_button.setToolTip("Pause or continue the game")
        self.pause_button.clicked.connect(self.tboard.pause)

        self.toolbar = self.addToolBar("MainToolBar")
        self.toolbar.setOrientation(Qt.Horizontal)
        self.toolbar.addWidget(self.reset_button)
        self.toolbar.addSeparator()
        self.toolbar.addWidget(self.pause_button)

        self.statusbar = self.statusBar()
        self.tboard.msg2StatusBar[str].connect(self.statusbar.showMessage)
        self.scoreBoard.make_connection(self.tboard)
        self.tboard.make_connection(self)
        self.tboard.connection(self.scoreBoard)

        self.tboard.start()

        self.resize(600, 600)
        self.center()
        self.setWindowTitle('DraughtsV3')
        self.show()

    def reset(self):
        '''
        Allow user to reset the game
        '''
        self.scoreBoard.p1_score = 0
        self.scoreBoard.p2_score = 0
        self.scoreBoard.p1_pieces_remaining = 12
        self.scoreBoard.p2_pieces_remaining = 12
        self.scoreBoard.label_player1.setText("Player1\n\nScore {}\n Remaining {}".format(self.scoreBoard.p1_score, self.scoreBoard.p1_pieces_remaining))
        self.scoreBoard.label_player2.setText("Player2\n\nScore {}\n Remaining {}".format(self.scoreBoard.p2_score, self.scoreBoard.p2_pieces_remaining))
        self.resetGame.emit()

    def get_board(self):
        return self.board

    def get_score_board(self):
        return self.scoreBoard

    def center(self):
        # centers the window on the screen
        screen = QDesktopWidget().screenGeometry()
        size = self.geometry()
        center_x = int((screen.width() - size.width()) / 2)
        center_y = int((screen.height() - size.height()) / 2)
        self.move(center_x, center_y)


if __name__ == "__main__":
    app = QApplication([])
    draughts = Draughts()
    sys.exit(app.exec_())


ModuleNotFoundError: No module named 'board'

In [5]:
'''piece.py'''

from enum import Enum


class Piece(Enum):
    NoPiece = 0
    Blue = 1
    Red = 2
    BlueCrown = 3
    RedCrown = 4


In [6]:
'''player.py'''

from enum import Enum


class Player(Enum):
    Player1 = 1
    Player2 = 2


In [7]:
'''scoreBoard.py'''

from PyQt5.QtWidgets import QDockWidget, QLabel, QWidget, QVBoxLayout, QMessageBox
from PyQt5.QtCore import pyqtSlot, Qt, pyqtSignal
import sys


class ScoreBoard(QDockWidget):
    winnerSignal = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        # initiates ScoreBoard
        self.player = 'Player1'
        self.p1_score = 0
        self.p2_score = 0
        self.p1_pieces_remaining = 12
        self.p2_pieces_remaining = 12
        self.resize(200, 200)
        self.setWindowTitle('ScoreBoard')
        self.widget = QWidget(self)
        self.widget.setMinimumSize(150,150) # Set minimum width to prevent text overflow
        self.main_layout = QVBoxLayout()

        # Update score widget with Player Specific scoring details
        self.label_player1 = QLabel("Player1\n\nScore {}\n Remaining {}".format(self.p1_score, self.p1_pieces_remaining))
        self.label_player1.setStyleSheet('')
        self.label_player1.setAlignment(Qt.AlignCenter | Qt.AlignVCenter)
        self.label_player1.setStyleSheet('font-weight: bold; background: rgb(128, 179, 255)')

        self.label_player2 = QLabel("Player2\n\nScore {}\n Remaining {}".format(self.p2_score, self.p2_pieces_remaining))
        self.label_player2.setStyleSheet('')
        self.label_player2.setAlignment(Qt.AlignCenter | Qt.AlignVCenter)

        self.p1_time_remaining = QLabel("Time\nRemaining:\n300")
        self.p1_time_remaining.setAlignment(Qt.AlignCenter | Qt.AlignVCenter)
        self.p1_time_remaining.setStyleSheet('font-weight: bold; background: rgb(128, 179, 255)')

        self.p2_time_remaining = QLabel("Time\nRemaining:\n300")
        self.p2_time_remaining.setAlignment(Qt.AlignCenter | Qt.AlignVCenter)

        self.widget.setLayout(self.main_layout)
        self.main_layout.addWidget(self.label_player1)
        self.main_layout.addWidget(self.p1_time_remaining)
        self.main_layout.addWidget(self.label_player2)
        self.main_layout.addWidget(self.p2_time_remaining)
        self.setWidget(self.widget)
        self.show()

    def make_connection(self, board):
        # this handles a signal sent from the board class
        # when the clickLocationSignal is emitted in board the set_click_location slot receives it
        # when the updateTimerSignal is emitted in the board the set_time_remaining slot receives it
        board.updateTimerSignal.connect(self.set_time_remaining)
        board.updateActivePlayer.connect(self.set_player_highlight)
        board.updateScore.connect(self.updateScore)

    @pyqtSlot(str)  # checks to make sure that the following slot is receiving an argument of the right type
    def updateScore(self, player):
        '''
        pyqtSlot from player class to update the score of the passed player
        '''
        if player == 'Player1':
            self.p1_score += 1
            self.p2_pieces_remaining -= 1
            # We need to update both players, 1 for score and the other for remaining pieces
            self.label_player1.setText("Player1\n\nScore {}\n Remaining {}".format(self.p1_score, self.p1_pieces_remaining))
            self.label_player2.setText("Player2\n\nScore {}\n Remaining {}".format(self.p2_score, self.p2_pieces_remaining))
            if self.p2_pieces_remaining == 0:
                self.winnerSignal.emit("Player1")

        elif player == 'Player2':
            self.p2_score += 1
            self.p1_pieces_remaining -= 1
            # We need to update both players, 1 for score and the other for remaining pieces
            self.label_player1.setText("Player1\n\nScore {}\n Remaining {}".format(self.p1_score, self.p1_pieces_remaining))
            self.label_player2.setText("Player2\n\nScore {}\n Remaining {}".format(self.p2_score, self.p2_pieces_remaining))
            if self.p1_pieces_remaining == 0:
                self.winnerSignal.emit("Player2")


    @pyqtSlot(int)
    def set_time_remaining(self, time_remaining):
        # updates the time remaining label to show the time remaining
        # Keep the changing value on a new line so as not to keep altering the width of the QDockWidget
        if self.player == 'Player1':
            update = "Time\nRemaining:\n" + str(time_remaining)
            self.p1_time_remaining.setText(update)

        elif self.player == 'Player2':
            update = "Time\nRemaining:\n" + str(time_remaining)
            self.p2_time_remaining.setText(update)

    @pyqtSlot(str)
    def set_player_highlight(self, player):
        '''
        pyqtSlot from player class to highlight the active player in the
        scoreBoard widget
        '''
        self.player = player

        if self.player == 'Player1':
            self.label_player1.setStyleSheet('font-weight: bold; background: rgb(128, 179, 255)')
            self.p1_time_remaining.setStyleSheet('font-weight: bold; background: rgb(128, 179, 255)')
            self.label_player2.setStyleSheet('')
            self.p2_time_remaining.setStyleSheet('')

        elif self.player == 'Player2':
            self.label_player2.setStyleSheet('font-weight: bold; background: rgb(255, 128, 128)')
            self.p2_time_remaining.setStyleSheet('font-weight: bold; background: rgb(255, 128, 128)')
            self.label_player1.setStyleSheet('')
            self.p1_time_remaining.setStyleSheet('')
        self.update()


## Design Decisions:  (task 10) 
We represented the player pieces with coloured ovals, red for one player and blue for the other. We used a visually appealing pastel colour pallet for all coloured elements of the project, including the yellow and green board (opposed to the traditional white and black). The scoreboard is plain and informational, but highlights information to indicate the current player. A statusbar was included to also indicate the current player, whether or not the game is paused and when a win state has been entered. A toolbar with a reset button and a pause/play button was placed at the top of the window, with appropriate icons and tooltips chosen for each button. The buttons are not explicitly named on-screen as the icons should be self-explanatory. Pieces are selected and moved with clicks and an error message is produced if and when an invalid square of the board is clicked at any time. When all pieces are captured or the timer reaches zero for a player, the other player wins and a message is presented with a reset button. The reset button will start a new game.

## Screen Shots of working Project x 5
![First](./img/start.png)

![Second](./img/piece_selected.png)

![Third](./img/mid_game.png)

![Fourth](./img/paused.png)

![Fifth](./img/win_state.png)